# **Homework 8 - Anomaly Detection**

If there are any questions, please contact mlta-2022spring-ta@googlegroups.com

Slide:    [Link]()　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw8)

# Set up the environment


## Package installation

In [59]:
# Training progress bar
# !pip install -q qqdm

## Downloading data

In [60]:
# !wget https://github.com/MachineLearningHW/HW8_Dataset/releases/download/v1.0.0/data.zip

In [61]:
# !unzip data.zip

# Import packages

In [62]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW, lr_scheduler
from qqdm import qqdm, format_str
import pandas as pd

# Loading data

In [63]:
train = np.load('data/trainingset.npy', allow_pickle=True)
test = np.load('data/testingset.npy', allow_pickle=True)

print(train.shape)
print(test.shape)

(100000, 64, 64, 3)
(19636, 64, 64, 3)


## Random seed
Set the random seed to a certain value for reproducibility.

In [64]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(48763)

# Autoencoder

# Models & loss

In [65]:
class fcn_autoencoder(nn.Module):
    def __init__(self):
        super(fcn_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 64 * 3, 2048),
            nn.LeakyReLU(),
            nn.Linear(2048, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 128),
#             nn.LeakyReLU(), 
#             nn.Linear(128, 32),
#             nn.LeakyReLU(),
#             nn.Linear(32, 16), 
#             nn.LeakyReLU(), 
#             nn.Linear(16, 8)
        )
        
        self.decoder = nn.Sequential(
#             nn.Linear(8, 16),
#             nn.LeakyReLU(), 
#             nn.Linear(16, 32),
#             nn.LeakyReLU(),
#             nn.Linear(32, 128),
#             nn.LeakyReLU(),
            nn.Linear(128, 512),
            nn.LeakyReLU(), 
            nn.Linear(512, 2048),
            nn.LeakyReLU(),
            nn.Linear(2048, 64 * 64 * 3), 
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(24, 48, 4, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),    
            nn.ReLU(),
        )
        self.enc_out_1 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),  
            nn.ReLU(),
        )
        self.enc_out_2 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1), 
            nn.Tanh(),
        )

    def encode(self, x):
        h1 = self.encoder(x)
        return self.enc_out_1(h1), self.enc_out_2(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


def loss_vae(recon_x, x, mu, logvar, criterion):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    mse = criterion(recon_x, x)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return mse + KLD

# Dataset module

Module for obtaining and processing data. The transform function here normalizes image's pixels from [0, 255] to [-1.0, 1.0].


In [66]:
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms."""
    def __init__(self, tensors):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
#             (100000, 3, 64, 64)
#             (19636, 3, 64, 64)
            self.tensors = tensors.permute(0, 3, 1, 2)
        
        self.transform = transforms.Compose([
          transforms.Lambda(lambda x: x.to(torch.float32)),
          transforms.Lambda(lambda x: 2. * x/255. - 1.),
        ])
        
    def __getitem__(self, index):
        x = self.tensors[index]
        
        if self.transform:
            # mapping images to [-1.0, 1.0]
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.tensors)

# Training

## Configuration


In [67]:
# Training hyperparameters
num_epochs = 50
batch_size = 256
learning_rate = 2e-3

# Build training dataloader
x = torch.from_numpy(train)
train_dataset = CustomTensorDataset(x)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Model
model_type = 'fcn'   # selecting a model type from {'cnn', 'fcn', 'vae', 'resnet'}
model_classes = {'fcn': fcn_autoencoder(), 'cnn': conv_autoencoder(), 'vae': VAE()}
model = model_classes[model_type].cuda()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_dataloader), epochs=num_epochs)

## Training loop

In [68]:
best_loss = np.inf
model.train()

qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'))
for epoch in qqdm_train:
    tot_loss = list()
    for data in train_dataloader:

        # ===================loading=====================
        img = data.float().cuda()
        if model_type in ['fcn']:
            img = img.view(img.shape[0], -1)

        # ===================forward=====================
        output = model(img)
        if model_type in ['vae']:
            loss = loss_vae(output[0], img, output[1], output[2], criterion)
        else:
            loss = criterion(output, img)

        tot_loss.append(loss.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
    # ===================save_best====================
    mean_loss = np.mean(tot_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))
    # ===================log========================
    qqdm_train.set_infos({
        'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
        'loss': f'{mean_loss:.4f}',
    })
    # ===================save_last========================
    torch.save(model, 'last_model_{}.pt'.format(model_type))

 Iters    Elapsed Time      Speed                                                                                       
 0/100          -             -                                                                                         
Description   0.0% |                                                                                                   | Iters    Elapsed Time      Speed    epoch   loss                                                                       
 1/100  00:00:12<00:21:23  0.08it/s  1/100  0.1399                                                                      
Description   1.0% |                                                                                                   | Iters    Elapsed Time      Speed    epoch   loss                                                                       
 2/100  00:00:26<00:21:26  0.08it/s  2/100  0.0837                                                                      
Description   2.0% |█             

Description  15.0% |██████████████                                                                                     | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 16/100  00:03:13<00:16:54  0.08it/s  16/100  0.0520                                                                    
Description  16.0% |███████████████                                                                                    | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 17/100  00:03:24<00:16:38  0.08it/s  17/100  0.0548                                                                    
Description  17.0% |████████████████                                                                                   | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 18/100  00:03:36<00:16:24  0.08it/

Description  27.0% |██████████████████████████                                                                         | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 28/100  00:05:43<00:14:42  0.08it/s  28/100  0.0348                                                                    
Description  28.0% |███████████████████████████                                                                        | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 29/100  00:05:57<00:14:34  0.08it/s  29/100  0.0344                                                                    
Description  29.0% |████████████████████████████                                                                       | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 30/100  00:06:11<00:14:25  0.08it/

Description  38.0% |█████████████████████████████████████                                                              | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 39/100  00:07:58<00:12:27  0.08it/s  39/100  0.0316                                                                    
Description  39.0% |██████████████████████████████████████                                                             | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 40/100  00:08:09<00:12:14  0.08it/s  40/100  0.0315                                                                    
Description  40.0% |███████████████████████████████████████                                                            | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 41/100  00:08:21<00:12:02  0.08it/

Description  47.0% |██████████████████████████████████████████████                                                     | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 48/100  00:09:47<00:10:36  0.08it/s  48/100  0.0302                                                                    
Description  48.0% |███████████████████████████████████████████████                                                    | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 49/100  00:10:00<00:10:25  0.08it/s  49/100  0.0300                                                                    
Description  49.0% |████████████████████████████████████████████████                                                   | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 50/100  00:10:13<00:10:13  0.08it/

...

Description  71.0% |██████████████████████████████████████████████████████████████████████                             | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 72/100  00:14:48<00:05:45  0.08it/s  72/100  0.0271                                                                    
Description  72.0% |███████████████████████████████████████████████████████████████████████                            | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 73/100  00:15:01<00:05:33  0.08it/s  73/100  0.0274                                                                    
Description  73.0% |████████████████████████████████████████████████████████████████████████                           | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 74/100  00:15:14<00:05:21  0.08it/

Description  78.0% |█████████████████████████████████████████████████████████████████████████████                      | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 79/100  00:16:21<00:04:20  0.08it/s  79/100  0.0265                                                                    
Description  79.0% |██████████████████████████████████████████████████████████████████████████████                     | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 80/100  00:16:34<00:04:08  0.08it/s  80/100  0.0265                                                                    
Description  80.0% |███████████████████████████████████████████████████████████████████████████████                    | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 81/100  00:16:48<00:03:56  0.08it/

Description  85.0% |████████████████████████████████████████████████████████████████████████████████████               | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 86/100  00:17:53<00:02:54  0.08it/s  86/100  0.0261                                                                    
Description  86.0% |█████████████████████████████████████████████████████████████████████████████████████              | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 87/100  00:18:07<00:02:42  0.08it/s  87/100  0.0261                                                                    
Description  87.0% |██████████████████████████████████████████████████████████████████████████████████████             | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 88/100  00:18:20<00:02:30  0.08it/

Description  91.0% |██████████████████████████████████████████████████████████████████████████████████████████         | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 92/100  00:19:13<00:01:40  0.08it/s  92/100  0.0259                                                                    
Description  92.0% |███████████████████████████████████████████████████████████████████████████████████████████        | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 93/100  00:19:26<00:01:27  0.08it/s  93/100  0.0259                                                                    
Description  93.0% |████████████████████████████████████████████████████████████████████████████████████████████       | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 94/100  00:19:38<00:01:15  0.08it/

Description  97.0% |████████████████████████████████████████████████████████████████████████████████████████████████   | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 98/100  00:20:30<00:00:25  0.08it/s  98/100  0.0258                                                                    
Description  98.0% |█████████████████████████████████████████████████████████████████████████████████████████████████  | Iters     Elapsed Time      Speed    epoch    loss                                                                     
 99/100  00:20:44<00:00:12  0.08it/s  99/100  0.0258                                                                    
Description  99.0% |██████████████████████████████████████████████████████████████████████████████████████████████████ |  Iters     Elapsed Time      Speed     epoch    loss                                                                   
 100/100  00:20:56<00:00:00  0.08it

# Inference
Model is loaded and generates its anomaly score predictions.

## Initialize
- dataloader
- model
- prediction file

In [69]:
eval_batch_size = 200

# build testing dataloader
data = torch.tensor(test, dtype=torch.float32)
test_dataset = CustomTensorDataset(data)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=eval_batch_size, num_workers=0)
eval_loss = nn.MSELoss(reduction='none')

# load trained model
checkpoint_path = f'last_model_{model_type}.pt'
model = torch.load(checkpoint_path)
model.eval()

# prediction file 
out_file = 'prediction.csv'

In [70]:
anomality = list()
with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        img = data.float().cuda()
        if model_type in ['fcn']:
            img = img.view(img.shape[0], -1)
        output = model(img)
        if model_type in ['vae']:
            output = output[0]
        if model_type in ['fcn']:
            loss = eval_loss(output, img).sum(-1)
        else:
            loss = eval_loss(output, img).sum([1, 2, 3])
        anomality.append(loss)
anomality = torch.cat(anomality, axis=0)
anomality = torch.sqrt(anomality).reshape(len(test), 1).cpu().numpy()

df = pd.DataFrame(anomality, columns=['score'])
df.to_csv(out_file, index_label = 'ID')